In [1]:
import pandas as pd


# Tool from text processing  
import nltk
from nltk.corpus import stopwords

# Tool  
import spacy
import pandas as pd

# Regual expressions
import re

df = pd.read_csv("OpenAi/amlo_clasify_chatpgt3.csv")

stop_words_es = stopwords.words('spanish')


nlp = spacy.load('es_core_news_lg')


def return_dataframe():

    df['cla_num'] = df['Clasificacion'].apply(clasification_to_num)
    df['Texto_limpio'] = df['Texto'].apply(clean_text)
    return df



# fn to clean text 

def clean_text(texto):
  textofin = texto.lower()
  textofin = re.sub(r'([^0-9A-Za-z-À-ÿ \t])','', textofin,)
  textofin = nlp(textofin)
  lema = []
  for token in textofin:
    lema.append(token.lemma_)

  textofin = lema
  textofin = ' '.join(textofin)
  return textofin
  
def clasification_to_num(text):
    if text == 'exterior':
        return 0
    elif text == 'economia':
        return 1
    elif text == 'opinion' :
        return 2
    elif text == 'competencia':
        return 3
    elif text == 'apoyo':
        return 4
    elif text == 'seguridad':
        return 5
  


df["clean"] = df["Texto"].apply(clean_text)



In [19]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
tfidf = TfidfVectorizer(ngram_range=(1, 3))

X = df["clean"]
y = df["classification_spanish"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.85, random_state=42
)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)
model = SVC(kernel="linear", random_state=30, probability=True)
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)

report = classification_report(y_test, y_pred, output_dict=True)
clasification = pd.DataFrame(report).transpose()


def predict_text(text):
    resultado = clean_text(text)
    resultado = tfidf.transform([resultado])
    prediccion = model.predict(resultado)
    probabilida = model.predict_proba(resultado)
    print(probabilida)
    return probabilida


def match_category(category):
    match category:
        case "0":
            return "exterior"
        case "1":
            return "economia"
        case "2":
            return "opinion"
        case "3":
            return "competencia"
        case "4":
            return "apoyo"
        case "5":
            return "seguridad"
        case _:
            return category
def match_category2(category):
    match category:
        case 0:
            return "exterior"
        case 1:
            return "economia"
        case 2:
            return "opinion"
        case 3:
            return "competencia"
        case 4:
            return "apoyo"
        case 5:
            return "seguridad"


def predict(proba):
    proba = list(proba[0])
    print(proba)
    maxx = max(proba)
    index = proba.index(maxx)
    return f"La probabilidad es {maxx} y lo categoriza como {index}"


def clasification_rep():
    clasification["Unnamed: 0"] = clasification["Unnamed: 0"].astype(str)

    clasification["Unnamed: 0"] = clasification["Unnamed: 0"].apply(match_category)
    a = clasification.rename(columns={"Unnamed: 0": "Clasificación", "precision" : "Precision"})

    return a


first = predict_text("Adultos mayores")

df.head()

print(predict(first))

print(clasification)




[[9.74860083e-01 2.55932154e-03 1.83062340e-03 5.64410391e-04
  8.98948111e-04 9.71460293e-04 4.74288001e-04 1.47523096e-02
  1.40037007e-04 1.66749169e-03 1.28102738e-03]]
[0.9748600825901733, 0.002559321537436722, 0.0018306234003551925, 0.0005644103910028399, 0.0008989481105962119, 0.0009714602934584963, 0.0004742880012612072, 0.014752309601579744, 0.0001400370067918859, 0.0016674916866736922, 0.0012810273806708058]
La probabilidad es 0.9748600825901733 y lo categoriza como 0
              precision    recall  f1-score      support
apoyo          0.465839  0.412088  0.437318   364.000000
competencia    0.404005  0.498547  0.446324   688.000000
construccion   0.628319  0.651376  0.639640   327.000000
corrupcion     0.522267  0.401869  0.454225   321.000000
economia       0.597403  0.544379  0.569659   169.000000
exterior       0.573333  0.403756  0.473829   213.000000
historia       0.562500  0.319527  0.407547   169.000000
opinion        0.436516  0.531290  0.479263   783.000000
salu

In [20]:

first = predict_text("Vamos, como lo hacemos todos lunes, a informar quién es quién en los precios, también el avance en las obras, el avance en la construcción del aeropuerto de Santa Lucía, el avance en la construcción de la refinería de Dos Bocas.")



print(predict(first))

[[4.04802358e-07 4.65235489e-07 9.99451418e-01 9.89789909e-06
  3.25592431e-04 2.14661747e-05 1.98501383e-05 1.30873649e-05
  6.89197121e-05 1.18848489e-05 7.70129315e-05]]
[4.048023582290229e-07, 4.6523548908093876e-07, 0.9994514184619914, 9.897899088706479e-06, 0.0003255924306322247, 2.1466174706003473e-05, 1.985013834271561e-05, 1.3087364908048738e-05, 6.891971211436844e-05, 1.1884848906935497e-05, 7.701293146257646e-05]
La probabilidad es 0.9994514184619914 y lo categoriza como 2
